LangChain Expression Language 를 사용해서 stuff chain을 구현해보자.

랭체인 구성요소

- Prompt
- Retriever
- LLM, ChatModel
- Tool
- OutputParser

우린 이미 Prompt, ChatModel, OutputParser에 대해 알아봤다.

Retriever에 대해 알아보자.

Retriever는 한 개의 string을 입력받는다.

질문이나 그와 관련성이 있는 document를 얻기위한 query를 입력받는다.

그리고 Retriever의 출력값은 document들의 List 이다.

입출력 타입을 안다면 구현(implement)은 쉽다.


먼저 구성 요소들의 순서를 생각해보자.

먼저 모든 document를 가져와야 한다. query와 관련있는 건 전부!

그렇다면 chain에 Retriever를 첫 번째로 넣어주면 되겠다.

retriver는 document들의 list를 반환한다.

그 document들은 template에 입력되어야 한다.

ChatPromptTemplate를 사용해서 chat messages를 사용하는 template를 만들어보자.

```python
# template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful assistant. answer question using only the following context. if you don't know the answer just say you don't know, don't make it up\n\n{context}"),
        ("human", "{question}"),
    ]
)
```

그 다음 구성요소는 prompt와 그 내부에 입력될 데이터를 전달받을 거야.

바로 LLM(ChatOpenAI).

```python
chain = retriever | prompt | llm
```

이제 chain을 실행해야 하는데, run이 아니라 invoke 메서드를 사용한다.

```python
chain.invoke("Describe Victory Mansions")
```

이걸 실행하면, 이 string("Describe Victory Mansions")이 retriever에게 전달될 것이다.

retriever 는 document들의 list를 반환할 거고

그 document들은 context값으로 입력되어야 하고

질문("Describe Victory Mansions")도 prompt template의 question으로 입력되어야 한다.


In [2]:
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate

# llm
llm = ChatOpenAI(temperature=0.1)
# 캐시 경로
cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
# .from_bytes_store 메서드는 임베딩 작업을 위해 필요한 embedder의 입력을 요구한다.
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

# vectorstore 변경
vectorstore = FAISS.from_documents(docs, embeddings)

# retriever 구현
retriever = vectorstore.as_retriever()

# template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            you are a helpful assistant. 
            answer question using only the following context. 
            if you don't know the answer just say you don't know. 
            don't make it up \n\n{context}
            """,
        ),
        ("human", "{question}"),
    ]
)

# retriver는 document들의 list를 반환한다.
chain = retriever | prompt | llm

chain.invoke("Describe Victory Mansions")

TypeError: list indices must be integers or slices, not str

당장 실행해 보면 코드가 작동하지 않는데, 그 이유는 우리가 아직 구체적인 동작을 구현하지 않았기 때문이다.

```python
chain = retriever | prompt | llm

chain.invoke("Describe Victory Mansions")
```

위에서 설명한 동작을 실제로 구현해봐야 한다.

우리가 원하는 동작은 질문값을 retriever에 전달하고 그 결과로 출력된 document들의 list를 prompt의 context에 입력하고

또 이 질문을 prompt의 question property(human의 question)로 전달하는 것 까지이다.

이 동작을 구현하기위해, 앞서 해본것과 굉장히 비슷한 작업을 할 거다.

{}를 사용해서 prompt의 context는 retriever로부터 오도록 한다.

```python
chain = {"context": retriever} | prompt | llm
```

전에 본 것처럼 retriever는 invoke에 입력해준 질문을 입력받아 호출될 것이다.

그리고 이 질문이 prompt의 question property로 전달되로록 해줘야 한다.

이건 RunnablePassthrough를 사용해서 아주 쉽게 할 수 있다.

```python
from langchain.schema.runnable import RunnablePassthrough
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm
```

RunnablePassthrough는 말 그대로 입력값이 통과하게(pass through) 해준다.

그러니까 질문(입력값)이 RunnablePassthrough()를 통해서 RunnablePassthrough()이 위치한 모든 곳들에 전달되도록 해준다.


In [3]:
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# llm
llm = ChatOpenAI(temperature=0.1)
# 캐시 경로
cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
# .from_bytes_store 메서드는 임베딩 작업을 위해 필요한 embedder의 입력을 요구한다.
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

# vectorstore 변경
vectorstore = FAISS.from_documents(docs, embeddings)

# retriever 구현
retriever = vectorstore.as_retriever()

# template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful assistant. answer question using only the following context. if you don't know the answer just say you don't know, don't make it up\n\n{context}"),
        ("human", "{question}"),
    ]
)

# retriver는 document들의 list를 반환한다.
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a building where Winston Smith resides. It is described as having glass doors, a hallway that smells of boiled cabbage and old rag mats, and being seven flights up. The building is depicted as having a faulty lift, with a large colored poster of a man\'s face with a caption that reads "BIG BROTHER IS WATCHING YOU" displayed indoors. The building is part of the cityscape of London, which is portrayed as a grim and colorless environment with rotting houses and oppressive government propaganda posters.')

실행해 보면 RunnablePassthrough() 덕분에 잘 동작하는 걸 확인할 수 있다.

Retriever는 chain의 아주 중요한 구성요소라는 것을 기억하자.

우린 Retriever를 chain의 구성요소로 사용할 수 있고,

Retriever는 한 개의 string을 입력받고 document들의 list를 출력해준다.

위에서 한 것은

```python
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)
```

LangChain이 질문("Describe Victory Mansions")을 retriever의 입력값으로 전달해서 호출해줬다.

```python
chain = (
    {
        "context": retriever("Describe Victory Mansions"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)
```

그럼 document들의 list가 반환되는 거다.

```python
chain = (
    {
        "context": [Doc],
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)
```

그리고 RunnablePassthrough는 이 입력을 prompt의 question에 전달해준다.

```python
chain = (
    {
        "context": retriever("Describe Victory Mansions"),
        "question": "Describe Victory Mansions",
    }
    | prompt
    | llm
)
```


그럼 document들의 list, 그리고 question, 이 두 가지가 prompt에 전달될 거다.

여기서 format_messages메서드를 호출하는 것과 같은 동작이 발생한다.

context값으로 document들이 들어가고, question도 옆에 입력될 것이다.

```python
chain = (
    {
        "context": retriever("Describe Victory Mansions"),
        "question": "Describe Victory Mansions",
    }
    | prompt.format_messages(context=context, qeustion=question,)
    | llm
)
```

그 다음으론 format된 prompt를 LLM에게 전달한다.
